In [7]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms

# Define the Body Measurement Estimation Network
class MeasureNet(nn.Module):
    def __init__(self, num_outputs=14):
        super(MeasureNet, self).__init__()
        
        # MNASNet backbone
        self.backbone = models.mnasnet1_0(pretrained=True)  # Using the depth multiplier of 1
        self.backbone.classifier = nn.Identity()  # Remove the classifier

        # MLP for final regression
        self.mlp = nn.Sequential(
            nn.Linear(1280 + 2, 128),  # 1280 features from MNASNet + 2 for height and weight
            nn.ReLU(),
            nn.Linear(128, num_outputs)  # 14 outputs for body measurements
        )
    
    def forward(self, x, height_weight):
        x = self.backbone(x)
        x = torch.cat([x, height_weight], dim=1)
        x = self.mlp(x)
        return x

# Create a random input tensor of size (batch_size, 3, 640, 960)
batch_size = 8
x = torch.randn(batch_size, 3, 640, 960)

# Create a random tensor for height and weight of size (batch_size, 2)
height_weight = torch.randn(batch_size, 2)

# Instantiate the network and print its architecture
model = MeasureNet()
# print(model)

# Forward pass through the network
output = model(x, height_weight)
print("Output shape:", output.shape)

Output shape: torch.Size([8, 14])


In [9]:
print(output[0])

tensor([ 0.0443,  0.0208, -0.0019, -0.1657, -0.0261,  0.0260,  0.0673, -0.0306,
         0.0863, -0.1025, -0.0430, -0.0318,  0.0179, -0.0159],
       grad_fn=<SelectBackward0>)
